In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

In [3]:
import snowflake.connector

In [4]:
ACCOUNT = 'bab48039'
USER = 'nzaw96'
PASSWORD = 'Abc123!@#'

## Seems like there's some issue with connecting Jupyter notebook to Snowflake. The goal of this connection was to directly export the processed files into tables in Snowflake. For now, the processed will be first save to my local computer and then staged+loaded into Snowflake using Snowsql.

In [5]:
# Seems like there's issues connecting Jupyter Notebook to Snowflake
con = snowflake.connector.connect(
  user=USER,
  password=PASSWORD,
  account=ACCOUNT,
)

MemoryError: Cannot allocate write+execute memory for ffi.callback(). You might be running on a system that prevents this. For more information, see https://cffi.readthedocs.io/en/latest/using.html#callbacks

## 1. Data is (are?) imported from Google's Covid-19 Open Data source. Each file is imported into a separate dataframe. The file 'index.csv' contains the keys that can be used to join the tables.

In [6]:
df_index = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/index.csv")
df_epidem = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/epidemiology.csv")
df_demog = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/demographics.csv")
df_emergDeclare = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/lawatlas-emergency-declarations.csv")
df_hospitalized = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/hospitalizations.csv")
df_mobility = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/mobility.csv")
df_vaccinations = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/vaccinations.csv")
df_govResponse =pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/oxford-government-response.csv")

## Now, we will clean all the dataframes in the order that they were imported in above.

### First up is df_index which is a dataframe that contains keys to join the tables together.

In [66]:
df_index.head()

,location_key,place_id,wikidata_id,datacommons_id,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,locality_code,locality_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level
0,AD,ChIJlfCemC71pRIRkn_qeNc-yQc,Q228,country/AND,AD,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,AD,AND,0
1,AE,ChIJvRKrsd9IXj4RpwoIwFYv0zM,Q878,country/ARE,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,AE,ARE,0
2,AF,ChIJbQL_-LZu0TgReNqWvg1GtfM,Q889,country/AFG,AF,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,AF,AFG,0
3,AF_BAL,ChIJWZOescNSyzgRkmVJ8fmfR3k,Q121104,NaN,AF,Afghanistan,BAL,Balkh,NaN,NaN,NaN,NaN,AF,AFG,1
4,AF_BAM,ChIJw-bKQwDr0jgRQiw8jHsQ-HI,Q171382,NaN,AF,Afghanistan,BAM,Bamyan,NaN,NaN,NaN,NaN,AF,AFG,1


### We will only keep rows where country_name = 'United States of America'

In [67]:
df_index = df_index.loc[df_index['country_name']=='United States of America']

### We also see that the values in the location_key column are specific to the city/county level as zipcodes are appended to them. We only want to specify the state so we can unappend the zipcode and cities. For example, 'US_FL_12009' -> 'US_FL'. So, we will add a column called 'location_key_2' that stores the state-level granularity of the location keys.

In [68]:
set(df_index['location_key'].tolist())

{'US_FL_12009',
 'US_OK_40107',
 'US_UT_49021',
 'US_AK_02195',
 'US_WV_54057',
 'US_WY_56015',
 'US_GA_13271',
 'US_MN_27109',
 'US_WY_56003',
 'US_CA_06035',
 'US_OK_40045',
 'US_GA',
 'US_GA_13251',
 'US_SC_45017',
 'US_FL_12127',
 'US_CO_08059',
 'US_IL_17055',
 'US_WV_54077',
 'US_SD_46081',
 'US_NJ_34013',
 'US_MN_27015',
 'US_MI_26079',
 'US_KS_20017',
 'US_IA_19017',
 'US_TX_48411',
 'US_MT_30029',
 'US_FL_12059',
 'US_CO_08071',
 'US_FL_12039',
 'US_TN_47089',
 'US_OK_40013',
 'US_FL_12037',
 'US_MO_29019',
 'US_ID_16083',
 'US_NJ_34037',
 'US_ID_16055',
 'US_KY_21139',
 'US_GA_13227',
 'US_CA_06087',
 'US_GA_13223',
 'US_AR_05015',
 'US_NM_35011',
 'US_FL_12053',
 'US_OH_39173',
 'US_KY_21073',
 'US_MT_30023',
 'US_PR_72123',
 'US_MI_26119',
 'US_MA_25017',
 'US_VT_50023',
 'US_OR_41019',
 'US_AK_02282',
 'US_VA_51107',
 'US_CA_06073',
 'US_OK_40071',
 'US_MT_30083',
 'US_TX_48383',
 'US_NY_36035',
 'US_VA_51640',
 'US_NE_31081',
 'US_KS_20057',
 'US_TN_47121',
 'US_GA_13183'

In [69]:
df_index['location_key_2'] = df_index['location_key'].apply(lambda x: '_'.join(x.split('_')[:2]))

In [597]:
df_index.head()

,location_key,place_id,wikidata_id,datacommons_id,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,locality_code,locality_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,location_key_2
19557,US,ChIJCzYy5IS16lQRQrfeQ5K5Oxw,Q30,country/USA,US,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,US,USA,0,US
19558,US_AK,ChIJG8CuwJzfAFQRNduKqSde27w,Q797,geoId/02,US,United States of America,AK,Alaska,NaN,NaN,NaN,NaN,US,USA,1,US_AK
19559,US_AK_02013,ChIJ7a-UgVuW_lYRjTetzbIWL48,Q504371,geoId/02013,US,United States of America,AK,Alaska,02013,Aleutians East,NaN,NaN,US,USA,2,US_AK
19560,US_AK_02016,ChIJI9ivbI1Pg1cRg8d4nlGjGVU,Q185533,geoId/02016,US,United States of America,AK,Alaska,02016,Aleutians West,NaN,NaN,US,USA,2,US_AK
19561,US_AK_02020,ChIJQT-zBHaRyFYR42iEp1q6fSU,Q39450,geoId/02020,US,United States of America,AK,Alaska,02020,Anchorage Municipality,NaN,NaN,US,USA,2,US_AK


### We will leave the df_index dataframe just as it is, at least for now. Let's now move on to df_epidem.

In [424]:
df_epidem = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/epidemiology.csv")

In [425]:
df_epidem.head()

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested
0,2020-01-01,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
1,2020-01-02,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
2,2020-01-03,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
3,2020-01-04,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
4,2020-01-05,AD,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN


### Just like with df_index, we will only keep the rows in df_epidem that correspond to the US. We will use location_key column to filter.

In [426]:
location_key_US = set(df_index['location_key'].tolist()) # this will be used later on as well for reference
df_epidem = df_epidem[df_epidem['location_key'].isin(location_key_US)]

In [427]:
df_epidem.head()

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested
9895200,2020-01-01,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
9895201,2020-01-02,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
9895202,2020-01-03,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
9895203,2020-01-04,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
9895204,2020-01-05,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN


### Just like what we did with df_index, let's add an additional column to the dataframe called 'location_key_2' which will display the state-level granularity of the location. For example, 'US_FL_12009' in 'location_key' will be 'US_FL' in 'location_key_2'.

In [428]:
df_epidem['location_key_2'] = df_epidem['location_key'].apply(lambda x: '_'.join(x.split('_')[:2]))

### We see that location_key_2 contains both just 'US' and 'US_stateX'. So, we will split df_epidem into two dataframes, df_epidem_US and df_epidem_states. 

In [430]:
df_epidem_2 = df_epidem.copy(deep=True)

In [431]:
df_epidem_2['date_month'] = df_epidem_2['date'].apply(lambda x: '-'.join(x.split('-')[:2]))

### Here, we will try to keep the rows where location_key is at the state_level. In other words, rows where location_key == location_key_2.

In [432]:
df_epidem_2.shape

(2551031, 12)

In [433]:
df_epidem_2 = df_epidem_2.loc[df_epidem_2['location_key'] == df_epidem_2['location_key_2']]

### Here's we will do groupby + fillna

In [434]:
df_epidem_2.head()

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested,location_key_2,date_month
9895200,2020-01-01,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,US,2020-01
9895201,2020-01-02,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,US,2020-01
9895202,2020-01-03,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,US,2020-01
9895203,2020-01-04,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,US,2020-01
9895204,2020-01-05,US,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,US,2020-01


In [435]:
df_epidem_2[['new_recovered', 'new_tested', 'new_deceased']] = df_epidem_2[['new_recovered', 'new_tested', 'new_deceased']].fillna(0)

In [436]:
colsForGroupbyFillna = ['cumulative_confirmed', 'cumulative_deceased', 'cumulative_recovered', 'cumulative_tested']
df_epidem_2[colsForGroupbyFillna] = df_epidem_2.groupby(['location_key'])[colsForGroupbyFillna].apply(lambda x: x.ffill())

In [439]:
df_epidem_2 = df_epidem_2.fillna(0)
df_epidem_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55206 entries, 9895200 to 12428419
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  55206 non-null  object 
 1   location_key          55206 non-null  object 
 2   new_confirmed         55206 non-null  float64
 3   new_deceased          55206 non-null  float64
 4   new_recovered         55206 non-null  float64
 5   new_tested            55206 non-null  float64
 6   cumulative_confirmed  55206 non-null  float64
 7   cumulative_deceased   55206 non-null  float64
 8   cumulative_recovered  55206 non-null  float64
 9   cumulative_tested     55206 non-null  float64
 10  location_key_2        55206 non-null  object 
 11  date_month            55206 non-null  object 
dtypes: float64(8), object(4)
memory usage: 5.5+ MB


### Since location_key and location_key_2 are exactly the same, we can drop location_key_2 since we don't need it anymore.

In [441]:
df_epidem_2.drop(columns=['location_key_2'], inplace=True)
df_epidem_2.head()

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested,date_month
9895200,2020-01-01,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-01
9895201,2020-01-02,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-01
9895202,2020-01-03,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-01
9895203,2020-01-04,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-01
9895204,2020-01-05,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020-01


In [442]:
df_epidem_US = df_epidem_2.loc[df_epidem_2['location_key'] == 'US']
df_epidem_US.reset_index(drop=True, inplace=True)
df_epidem_states = df_epidem_2.loc[df_epidem_2['location_key'] != 'US']
df_epidem_states.reset_index(drop=True, inplace=True)

### We see that there is one row in the dataframe where data_month is '2022-12' (and date is '2022-12-30') which is most likely a mistake and should be '2022-08', because the data source states the data was last actively added to the repo on Sep 15, 2022.

In [443]:
df_epidem_states.loc[df_epidem_states['date_month'] == '2022-12']

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested,date_month
8712,2022-12-30,US_DC,-31136.0,0.0,0.0,0.0,103487.0,1382.0,34985.0,1261363.0,2022-12


In [444]:
df_epidem_states['date'] = df_epidem_states['date'].replace(['2022-12-30'], '2022-08-30')
df_epidem_states['date_month'] = df_epidem_states['date_month'].replace(['2022-12'], '2022-08')

/var/folders/8_/94pckqjd52dgrgd_fxmw31280000gp/T/ipykernel_3674/3135985351.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epidem_states['date'] = df_epidem_states['date'].replace(['2022-12-30'], '2022-08-30')
/var/folders/8_/94pckqjd52dgrgd_fxmw31280000gp/T/ipykernel_3674/3135985351.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epidem_states['date_month'] = df_epidem_states['date_month'].replace(['2022-12'], '2022-08')


### Another thing we see is that columns that should be int are currently float. We need to cast them back to the correct data type.

In [445]:
def fixDataTypesOfColumns(df, colsToChange, typeFrom, typeTo):
    for col in colsToChange:
        if df[col].dtypes == typeFrom:
            df[col] = df[col].astype(typeTo)
        else:
            print('Datatypes not matching!')

In [447]:
df_epidem_US.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988 entries, 0 to 987
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  988 non-null    object 
 1   location_key          988 non-null    object 
 2   new_confirmed         988 non-null    float64
 3   new_deceased          988 non-null    float64
 4   new_recovered         988 non-null    float64
 5   new_tested            988 non-null    float64
 6   cumulative_confirmed  988 non-null    float64
 7   cumulative_deceased   988 non-null    float64
 8   cumulative_recovered  988 non-null    float64
 9   cumulative_tested     988 non-null    float64
 10  date_month            988 non-null    object 
dtypes: float64(8), object(3)
memory usage: 85.0+ KB


In [448]:
fixDataTypesOfColumns(df_epidem_states, df_epidem_states.columns.tolist()[2:10], 'float64', 'int')
fixDataTypesOfColumns(df_epidem_US, df_epidem_US.columns.tolist()[2:10], 'float64', 'int')

/var/folders/8_/94pckqjd52dgrgd_fxmw31280000gp/T/ipykernel_3674/4103920634.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(typeTo)


### Now, moving on to the demographics data (df_demog).

In [449]:
df_demog = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/demographics.csv")

In [450]:
df_demog.head()

,location_key,population,population_male,population_female,population_rural,population_urban,population_largest_city,population_clustered,population_density,human_development_index,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older
0,AD,77265.0,58625.0,55581.0,9269.0,67873.0,NaN,NaN,164.394,0.858,9370.0,12022.0,10727.0,12394.0,21001.0,20720.0,14433.0,8657.0,4881.0
1,AE,9890400.0,6836349.0,3054051.0,1290785.0,8479744.0,2833079.0,5914068.0,118.306,0.863,1011713.0,842991.0,2149343.0,3169314.0,1608109.0,797913.0,242707.0,55884.0,12426.0
2,AF,38928341.0,19976265.0,18952076.0,28244481.0,9797273.0,4114030.0,4114030.0,59.627,0.498,11088732.0,9821559.0,7035871.0,4534646.0,2963459.0,1840198.0,1057496.0,480455.0,105925.0
3,AF_BAL,1509183.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AF_BAM,495557.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Again, here we will only keep the rows that correspond to the US.

In [451]:
df_demog = df_demog[df_demog.location_key.isin(location_key_US)]

In [452]:
df_demog['location_key_2'] = df_demog['location_key'].apply(lambda x: '_'.join(x.split('_')[:2]))

In [453]:
df_demog.reset_index(drop=True, inplace=True)
df_demog.head()

,location_key,population,population_male,population_female,population_rural,population_urban,population_largest_city,population_clustered,population_density,human_development_index,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older,location_key_2
0,US,331002647.0,163786016.0,167216631.0,57576495.0,270663028.0,18804722.0,152587187.0,36.185,0.92,39721484.0,42332395.0,46094075.0,44668270.0,40348397.0,42120077.0,38488170.0,24082597.0,13147182.0,US
1,US_AK,733391.0,424916.0,391925.0,NaN,NaN,NaN,NaN,NaN,NaN,115198.0,111045.0,128022.0,113254.0,102310.0,116061.0,82608.0,35021.0,13322.0,US_AK
2,US_AK_02013,3455.0,2345.0,1110.0,NaN,NaN,NaN,NaN,NaN,NaN,178.0,165.0,571.0,532.0,792.0,696.0,321.0,156.0,44.0,US_AK
3,US_AK_02016,5729.0,3795.0,1934.0,NaN,NaN,NaN,NaN,NaN,NaN,359.0,400.0,951.0,957.0,1149.0,1146.0,622.0,118.0,27.0,US_AK
4,US_AK_02020,294338.0,149817.0,144521.0,NaN,NaN,NaN,NaN,NaN,NaN,41514.0,37461.0,49984.0,45570.0,35093.0,37143.0,28931.0,12887.0,5755.0,US_AK


In [454]:
df_demog_US = df_demog.loc[df_demog['location_key'] == 'US']

In [455]:
df_demog_states = df_demog.loc[df_demog['location_key'] != 'US']
df_demog_states.reset_index(drop=True, inplace=True)

In [456]:
df_demog_states.head()

,location_key,population,population_male,population_female,population_rural,population_urban,population_largest_city,population_clustered,population_density,human_development_index,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older,location_key_2
0,US_AK,733391.0,424916.0,391925.0,NaN,NaN,NaN,NaN,NaN,NaN,115198.0,111045.0,128022.0,113254.0,102310.0,116061.0,82608.0,35021.0,13322.0,US_AK
1,US_AK_02013,3455.0,2345.0,1110.0,NaN,NaN,NaN,NaN,NaN,NaN,178.0,165.0,571.0,532.0,792.0,696.0,321.0,156.0,44.0,US_AK
2,US_AK_02016,5729.0,3795.0,1934.0,NaN,NaN,NaN,NaN,NaN,NaN,359.0,400.0,951.0,957.0,1149.0,1146.0,622.0,118.0,27.0,US_AK
3,US_AK_02020,294338.0,149817.0,144521.0,NaN,NaN,NaN,NaN,NaN,NaN,41514.0,37461.0,49984.0,45570.0,35093.0,37143.0,28931.0,12887.0,5755.0,US_AK
4,US_AK_02050,18090.0,9369.0,8721.0,NaN,NaN,NaN,NaN,NaN,NaN,3888.0,3211.0,2833.0,2294.0,1682.0,2053.0,1378.0,568.0,183.0,US_AK


### We see that the six of the columns in df_demog_states, namely population_rural, population_urban, population_largest_city, population_clustered, population_density  and human_development_index, have NaN values in all the rows. This means that these are the columns only useful in comparing different countries. Since we are limiting our analyses to the US, we can drop all six columns.

In [457]:
colsDrop_dfDemogStates = ['population_rural', 'population_urban', 'population_largest_city', 'population_clustered',
                           'population_density', 'human_development_index']
df_demog_states = df_demog_states.drop(columns=colsDrop_dfDemogStates)

In [458]:
df_demog_states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3284 entries, 0 to 3283
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   location_key                 3284 non-null   object 
 1   population                   3284 non-null   float64
 2   population_male              3202 non-null   float64
 3   population_female            3202 non-null   float64
 4   population_age_00_09         3202 non-null   float64
 5   population_age_10_19         3202 non-null   float64
 6   population_age_20_29         3202 non-null   float64
 7   population_age_30_39         3202 non-null   float64
 8   population_age_40_49         3202 non-null   float64
 9   population_age_50_59         3202 non-null   float64
 10  population_age_60_69         3202 non-null   float64
 11  population_age_70_79         3202 non-null   float64
 12  population_age_80_and_older  3202 non-null   float64
 13  location_key_2    

### There seems to be 82 rows where all the columns, except for location_key and location_key_2, are null. We will go ahead and drop those rows.

In [459]:
df_demog_states.loc[df_demog_states['population_male'].isna()]

,location_key,population,population_male,population_female,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older,location_key_2
174,US_AS,49710.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US_AS
1546,US_MP_69110,48220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US_MP
1547,US_MP_69120,3136.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US_MP
2359,US_PR_72001,18020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US_PR
2360,US_PR_72003,38136.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US_PR
2361,US_PR_72005,55101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US_PR
2362,US_PR_72007,24223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US_PR
2363,US_PR_72009,24637.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US_PR
2364,US_PR_72011,25596.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US_PR
2365,US_PR_72013,87754.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US_PR


In [460]:
df_demog_states = df_demog_states.dropna()
df_demog_states.reset_index(drop=True, inplace=True)

In [461]:
fixDataTypesOfColumns(df_demog_states, df_demog_states.columns.tolist()[1:-1], 'float64', 'int64')

In [462]:
fixDataTypesOfColumns(df_demog_US, df_demog_US.columns.tolist()[1:-1], 'float64', 'int64')

/var/folders/8_/94pckqjd52dgrgd_fxmw31280000gp/T/ipykernel_3674/4103920634.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(typeTo)


In [463]:
df_demog_states.head()

,location_key,population,population_male,population_female,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older,location_key_2
0,US_AK,733391,424916,391925,115198,111045,128022,113254,102310,116061,82608,35021,13322,US_AK
1,US_AK_02013,3455,2345,1110,178,165,571,532,792,696,321,156,44,US_AK
2,US_AK_02016,5729,3795,1934,359,400,951,957,1149,1146,622,118,27,US_AK
3,US_AK_02020,294338,149817,144521,41514,37461,49984,45570,35093,37143,28931,12887,5755,US_AK
4,US_AK_02050,18090,9369,8721,3888,3211,2833,2294,1682,2053,1378,568,183,US_AK


### As with df_epidem, we will only keep the rows where location_key == location_key_2 since we only want to retain data at the state level. Then, we will drop location_key_2 column.

In [465]:
df_demog_states = df_demog_states.loc[df_demog_states['location_key'] == df_demog_states['location_key_2']]

In [468]:
df_demog_US.drop(columns=['location_key_2'], inplace=True)
df_demog_states.drop(columns=['location_key_2'], inplace=True)

/var/folders/8_/94pckqjd52dgrgd_fxmw31280000gp/T/ipykernel_3674/1539207519.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_demog_US.drop(columns=['location_key_2'], inplace=True)


### Moving onto df_emergDeclare, we see that there are 104 rows in df_emergDeclare. We will only hand-pick some and retain them.

In [471]:
pd.set_option('display.max_columns', None)
df_emergDeclare.tail(10)

,date,location_key,lawatlas_mitigation_policy,lawatlas_state_emergency,lawatlas_emerg_statewide,lawatlas_travel_requirement,lawatlas_traveler_type_all_people_entering_the_state,lawatlas_traveler_type_travelers_from_specified_states,lawatlas_traveler_type_travelers_from_specified_countries,lawatlas_traveler_type_general_international_travelers,lawatlas_traveler_type_all_air_travelers,lawatlas_requirement_type_traveler_must_self_quarantine,lawatlas_requirement_type_traveler_must_inform_others_of_travel,lawatlas_requirement_type_checkpoints_must_be_established,lawatlas_requirement_type_travel_requirement_must_be_posted,lawatlas_travel_statewide,lawatlas_home_requirement,lawatlas_home_except_engaging_in_essential_business_activities,lawatlas_home_except_obtaining_necessary_supplies,lawatlas_home_except_accessing_emergency_services,lawatlas_home_except_caring_for_a_person_outside_the_home,lawatlas_home_except_caring_for_a_pet_outside_the_home,lawatlas_home_except_engaging_in_outdoor_activities,lawatlas_home_except_attending_religious_services,lawatlas_home_except_engaging_in_essential_health_care_services,lawatlas_home_statewide,lawatlas_curfew_reg,lawatlas_mask_requirement,lawatlas_mask_statewide,lawatlas_business_close,lawatlas_business_type_all_non_essential_businesses,lawatlas_business_type_non_essential_retail_businesses,lawatlas_business_type_entertainment_businesses,lawatlas_business_type_personal_service_businesses,lawatlas_business_type_restaurants,lawatlas_business_type_bars,lawatlas_business_type_fitness_centers,lawatlas_essential_def_appliance_stores,lawatlas_essential_def_convenience_stores,lawatlas_essential_def_gas_stations,lawatlas_essential_def_grocery_stores,lawatlas_essential_def_gun_stores,lawatlas_essential_def_hardware_stores,lawatlas_essential_def_liquor_stores,lawatlas_essential_def_pharmacies,lawatlas_essential_def_marijuana_dispensaries,lawatlas_essential_def_pet_stores,lawatlas_essential_def_type_of_essential_retail_business_is_not_specified,lawatlas_essential_def_no_restriction_on_retail_businesses,lawatlas_business_statewide,lawatlas_rest_restrict,lawatlas_service_type_takeout,lawatlas_service_type_delivery,lawatlas_service_type_limited_on_site_service,lawatlas_rest_statewide,lawatlas_schools_requirement,lawatlas_schools_type_private_elementary_schools,lawatlas_schools_type_private_secondary_schools,lawatlas_schools_type_public_elementary_schools,lawatlas_schools_type_public_secondary_schools,lawatlas_schools_type_colleges_and_universities,lawatlas_schools_type_technical_schools,lawatlas_schools_type_type_of_school_not_specified,lawatlas_schools_statewide,lawatlas_gathering_ban,lawatlas_gathering_type,lawatlas_gathering_statewide,lawatlas_med_restrict,lawatlas_med_except_delay_would_threaten_patients_health,lawatlas_med_except_delay_would_threaten_patients_life,lawatlas_med_except_procedure_needed_to_treat_emergency,lawatlas_med_except_procedure_does_not_deplete_hospital_capacity,lawatlas_med_except_procedure_does_not_deplete_personal_protective_equipment,lawatlas_med_except_family_planning_services,lawatlas_med_except_no_exception_specified,lawatlas_abortion_essential_new,lawatlas_med_statewide,lawatlas_correct_requirement,lawatlas_correct_facility_all_state_facilities,lawatlas_correct_facility_all_department_of_corrections_facilities,lawatlas_correct_facility_all_county_jails,lawatlas_correct_facility_juvenile_detention_centers,lawatlas_correct_type_intakes_suspended,lawatlas_correct_type_duty_to_receive_prisoners_suspended,lawatlas_correct_type_transfers_to_custody_suspended,lawatlas_correct_type_release_of_inmates,lawatlas_correct_type_rules_regarding_inmate_release_suspended,lawatlas_correct_type_release_notice_suspended,lawatlas_correct_type_cease_in_person_parole_hearings,lawatlas_correct_type_develop_process_for_virtual_parole_hearings,lawatlas_correct_type_visitation_suspended,lawatlas_correct_statewide,lawatlas_state_preempt,lawatlas_action_preempt_imposing_additional_social_distancing_limitat

In [472]:
la = 'lawatlas_' # this appears in every column name
df_emergD2 = df_emergDeclare[['date', 'location_key', la+'mitigation_policy', la+'emerg_statewide', 
                              la+'requirement_type_traveler_must_self_quarantine',
                             la+'home_statewide', la+'mask_statewide',
                             la+'schools_statewide', la+'gathering_statewide']]

### Next, we see that the values in all the columns except for the data and location_key columns are either integers 0 or 1. We need to change it from float to int type. For that, first we will replace all the NaN values with 0.

In [473]:
df_emergD2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8364 entries, 0 to 8363
Data columns (total 9 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   date                                                     8364 non-null   object 
 1   location_key                                             8364 non-null   object 
 2   lawatlas_mitigation_policy                               8364 non-null   int64  
 3   lawatlas_emerg_statewide                                 5794 non-null   float64
 4   lawatlas_requirement_type_traveler_must_self_quarantine  0 non-null      float64
 5   lawatlas_home_statewide                                  2157 non-null   float64
 6   lawatlas_mask_statewide                                  2325 non-null   float64
 7   lawatlas_schools_statewide                               4602 non-null   float64
 8   lawatlas_gathering_statewide 

### Seeing that all the entries in '...traveler_must_self_quarantine' column are NaN, we can drop that column entirely.

In [474]:
df_emergD2.drop(columns=[la+'requirement_type_traveler_must_self_quarantine'], inplace=True)

/var/folders/8_/94pckqjd52dgrgd_fxmw31280000gp/T/ipykernel_3674/4259409994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emergD2.drop(columns=[la+'requirement_type_traveler_must_self_quarantine'], inplace=True)


In [475]:
df_emergD2.reset_index(drop=True, inplace=True)

### Looks like we don't need to create location_key_2 column since entries in location_key are already at the state-level granularity.

In [476]:
set(df_emergD2['location_key'].tolist())

{'US_AK',
 'US_AL',
 'US_AR',
 'US_AZ',
 'US_CA',
 'US_CO',
 'US_CT',
 'US_DC',
 'US_DE',
 'US_FL',
 'US_GA',
 'US_HI',
 'US_IA',
 'US_ID',
 'US_IL',
 'US_IN',
 'US_KS',
 'US_KY',
 'US_LA',
 'US_MA',
 'US_MD',
 'US_ME',
 'US_MI',
 'US_MN',
 'US_MO',
 'US_MS',
 'US_MT',
 'US_NC',
 'US_ND',
 'US_NE',
 'US_NH',
 'US_NJ',
 'US_NM',
 'US_NV',
 'US_NY',
 'US_OH',
 'US_OK',
 'US_OR',
 'US_PA',
 'US_RI',
 'US_SC',
 'US_SD',
 'US_TN',
 'US_TX',
 'US_UT',
 'US_VA',
 'US_VT',
 'US_WA',
 'US_WI',
 'US_WV',
 'US_WY'}

### Now, let's fill the NaN values using ffill first. Then we will fill the rest of the NaN values with 0.

In [477]:
df_emergD2 = df_emergD2.groupby(['location_key']).apply(lambda x: x.ffill())
df_emergD2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8364 entries, 0 to 8363
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   date                          8364 non-null   object 
 1   location_key                  8364 non-null   object 
 2   lawatlas_mitigation_policy    8364 non-null   int64  
 3   lawatlas_emerg_statewide      5794 non-null   float64
 4   lawatlas_home_statewide       3766 non-null   float64
 5   lawatlas_mask_statewide       2383 non-null   float64
 6   lawatlas_schools_statewide    4952 non-null   float64
 7   lawatlas_gathering_statewide  5008 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 588.1+ KB


In [479]:
df_emergD2 = df_emergD2.fillna(0)
df_emergD2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8364 entries, 0 to 8363
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   date                          8364 non-null   object 
 1   location_key                  8364 non-null   object 
 2   lawatlas_mitigation_policy    8364 non-null   int64  
 3   lawatlas_emerg_statewide      8364 non-null   float64
 4   lawatlas_home_statewide       8364 non-null   float64
 5   lawatlas_mask_statewide       8364 non-null   float64
 6   lawatlas_schools_statewide    8364 non-null   float64
 7   lawatlas_gathering_statewide  8364 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 588.1+ KB


### Looks like all the rows in df_emergD2 are at the state-level granularity and there's none at the country-level granularity. Therefore, we DON'T need to split df_emergD2 into df_emergD2_US and df_emergD2_states.

In [480]:
df_emergD2.loc[df_emergD2['location_key'] == 'US'].shape

(0, 8)

In [481]:
fixDataTypesOfColumns(df_emergD2, df_emergD2.columns.tolist()[3:], 'float64', 'int64')

In [484]:
df_emergD2['date_month'] = df_emergD2['date'].apply(lambda x: '-'.join(x.split('-')[:2]))

### Moving onto df_hospitalized

In [493]:
df_hospitalized = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/hospitalizations.csv")

In [494]:
df_hospitalized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1768485 entries, 0 to 1768484
Data columns (total 11 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   date                                object 
 1   location_key                        object 
 2   new_hospitalized_patients           float64
 3   cumulative_hospitalized_patients    float64
 4   current_hospitalized_patients       float64
 5   new_intensive_care_patients         float64
 6   cumulative_intensive_care_patients  float64
 7   current_intensive_care_patients     float64
 8   new_ventilator_patients             float64
 9   cumulative_ventilator_patients      float64
 10  current_ventilator_patients         float64
dtypes: float64(9), object(2)
memory usage: 148.4+ MB


In [496]:
df_hospitalized = df_hospitalized[df_hospitalized['location_key'].isin(location_key_US)]

In [497]:
df_hospitalized.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58141 entries, 1710344 to 1768484
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   date                                58141 non-null  object 
 1   location_key                        58141 non-null  object 
 2   new_hospitalized_patients           57238 non-null  float64
 3   cumulative_hospitalized_patients    53093 non-null  float64
 4   current_hospitalized_patients       49380 non-null  float64
 5   new_intensive_care_patients         4521 non-null   float64
 6   cumulative_intensive_care_patients  4209 non-null   float64
 7   current_intensive_care_patients     47186 non-null  float64
 8   new_ventilator_patients             1705 non-null   float64
 9   cumulative_ventilator_patients      1710 non-null   float64
 10  current_ventilator_patients         9546 non-null   float64
dtypes: float64(9), object(2)
memory u

In [501]:
df_hospitalized['location_key_2'] = df_hospitalized['location_key'].apply(lambda x: '_'.join(x.split('_')[:2]))

In [504]:
df_hospitalized = df_hospitalized.loc[df_hospitalized['location_key'] == df_hospitalized['location_key_2']]
df_hospitalized.drop(columns=['location_key_2'], inplace=True)
df_hospitalized.shape

(51676, 11)

### Now, we need to do fillna(0) for the 'new_...' columns and groupby + fillna(ffill) for the 'cumulative_...' columns.

In [505]:
df_hospitalized.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51676 entries, 1710344 to 1768484
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   date                                51676 non-null  object 
 1   location_key                        51676 non-null  object 
 2   new_hospitalized_patients           51676 non-null  float64
 3   cumulative_hospitalized_patients    47531 non-null  float64
 4   current_hospitalized_patients       48477 non-null  float64
 5   new_intensive_care_patients         4521 non-null   float64
 6   cumulative_intensive_care_patients  4209 non-null   float64
 7   current_intensive_care_patients     46283 non-null  float64
 8   new_ventilator_patients             1705 non-null   float64
 9   cumulative_ventilator_patients      1710 non-null   float64
 10  current_ventilator_patients         9546 non-null   float64
dtypes: float64(9), object(2)
memory u

In [506]:
colsForFillZero = ['new_hospitalized_patients', 'new_intensive_care_patients', 'new_ventilator_patients']
colsForGroupbyFfill = ['cumulative_hospitalized_patients', 'cumulative_intensive_care_patients', 
                      'cumulative_ventilator_patients', 'current_hospitalized_patients', 
                       'current_intensive_care_patients', 'current_ventilator_patients']
df_hospitalized[colsForFillZero] = df_hospitalized[colsForFillZero].fillna(0)
df_hospitalized[colsForGroupbyFfill] = df_hospitalized.groupby(['location_key'])[colsForGroupbyFfill].apply(lambda x: x.ffill())

In [508]:
df_hospitalized = df_hospitalized.fillna(0)

In [511]:
df_hospitalized['date_month'] = df_hospitalized['date'].apply(lambda x: '-'.join(x.split('-')[:2]))

In [512]:
df_hosp_US = df_hospitalized.loc[df_hospitalized['location_key'] == 'US']
df_hosp_states = df_hospitalized.loc[df_hospitalized['location_key'] != 'US']

In [516]:
df_hosp_states.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50699 entries, 1711321 to 1768484
Data columns (total 12 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   date                                50699 non-null  object 
 1   location_key                        50699 non-null  object 
 2   new_hospitalized_patients           50699 non-null  float64
 3   cumulative_hospitalized_patients    50699 non-null  float64
 4   current_hospitalized_patients       50699 non-null  float64
 5   new_intensive_care_patients         50699 non-null  float64
 6   cumulative_intensive_care_patients  50699 non-null  float64
 7   current_intensive_care_patients     50699 non-null  float64
 8   new_ventilator_patients             50699 non-null  float64
 9   cumulative_ventilator_patients      50699 non-null  float64
 10  current_ventilator_patients         50699 non-null  float64
 11  date_month                       

In [517]:
df_hosp_US.reset_index(drop=True, inplace=True)
df_hosp_states.reset_index(drop=True, inplace=True)
fixDataTypesOfColumns(df_hosp_US, df_hosp_US.columns.tolist()[2:11], 'float64', 'int64')
fixDataTypesOfColumns(df_hosp_states, df_hosp_states.columns.tolist()[2:11], 'float64', 'int64')

/var/folders/8_/94pckqjd52dgrgd_fxmw31280000gp/T/ipykernel_3674/4103920634.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(typeTo)


### Next is df_mobility

In [520]:
df_mobility = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/mobility.csv")

In [521]:
df_mobility.head()

,date,location_key,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential
0,2020-02-15,AE,0.0,4.0,5.0,0.0,2.0,1.0
1,2020-02-16,AE,1.0,4.0,4.0,1.0,2.0,1.0
2,2020-02-17,AE,-1.0,1.0,5.0,1.0,2.0,1.0
3,2020-02-18,AE,-2.0,1.0,5.0,0.0,2.0,1.0
4,2020-02-19,AE,-2.0,0.0,4.0,-1.0,2.0,1.0


In [522]:
df_mobility = df_mobility[df_mobility['location_key'].isin(location_key_US)]

In [523]:
df_mobility.isna().sum()

date                                    0
location_key                            0
mobility_retail_and_recreation     860673
mobility_grocery_and_pharmacy     1024790
mobility_parks                    1846480
mobility_transit_stations         1522028
mobility_workplaces                 44522
mobility_residential              1009563
dtype: int64

In [526]:
df_mobility['location_key_2'] = df_mobility['location_key'].apply(lambda x: '_'.join(x.split('_')[:2]))

In [527]:
df_mobility.shape

(2445945, 9)

In [528]:
df_mobility = df_mobility.loc[df_mobility['location_key'] == df_mobility['location_key_2']]
df_mobility.drop(columns=['location_key_2'], inplace=True)
df_mobility.shape

(48932, 8)

In [529]:
df_mobility.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48932 entries, 3845350 to 6274306
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   date                            48932 non-null  object 
 1   location_key                    48932 non-null  object 
 2   mobility_retail_and_recreation  48932 non-null  float64
 3   mobility_grocery_and_pharmacy   48932 non-null  float64
 4   mobility_parks                  48392 non-null  float64
 5   mobility_transit_stations       48797 non-null  float64
 6   mobility_workplaces             48932 non-null  float64
 7   mobility_residential            48932 non-null  float64
dtypes: float64(6), object(2)
memory usage: 3.4+ MB


In [530]:
df_mobility = df_mobility.groupby(['location_key']).apply(lambda x: x.ffill())

In [532]:
df_mob_US = df_mobility.loc[df_mobility['location_key'] == 'US']
df_mob_states = df_mobility.loc[df_mobility['location_key'] != 'US']

### Next we look at df_vaccinations

In [534]:
df_vaccinations = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/vaccinations.csv")

In [535]:
df_vaccinations = df_vaccinations[df_vaccinations['location_key'].isin(location_key_US)]

In [539]:
df_vaccinations['location_key_2'] = df_vaccinations['location_key'].apply(lambda x: '_'.join(x.split('_')[:2]))

In [541]:
df_vaccinations = df_vaccinations.loc[df_vaccinations['location_key'] == df_vaccinations['location_key_2']]
df_vaccinations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34122 entries, 916955 to 2531126
Data columns (total 33 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   date                                           34122 non-null  object 
 1   location_key                                   34122 non-null  object 
 2   new_persons_vaccinated                         33968 non-null  float64
 3   cumulative_persons_vaccinated                  34025 non-null  float64
 4   new_persons_fully_vaccinated                   33947 non-null  float64
 5   cumulative_persons_fully_vaccinated            33955 non-null  float64
 6   new_vaccine_doses_administered                 34065 non-null  float64
 7   cumulative_vaccine_doses_administered          34122 non-null  float64
 8   new_persons_vaccinated_pfizer                  0 non-null      float64
 9   cumulative_persons_vaccinated_pfizer       

### Next, we see that some of the columns are specific to the a brand of vaccine. For our purporses, we won't differentiate between pfizer, moderna and janssen so we will not keep those columns that have the name of a covid vaccine brand in their names.

In [542]:
df_vacc = df_vaccinations[['date', 'location_key', 
                          'new_persons_vaccinated',
                          'cumulative_persons_vaccinated',
                          'new_persons_fully_vaccinated',
                          'cumulative_persons_fully_vaccinated',
                          'new_vaccine_doses_administered',
                          'cumulative_vaccine_doses_administered']]

In [543]:
df_vacc['date_month'] = df_vacc['date'].apply(lambda x: '-'.join(x.split('-')[:2]))
df_vacc.head()

/var/folders/8_/94pckqjd52dgrgd_fxmw31280000gp/T/ipykernel_3674/13640423.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vacc['date_month'] = df_vacc['date'].apply(lambda x: '-'.join(x.split('-')[:2]))


,date,location_key,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered,date_month
916955,2020-12-13,US,30817.0,30817.0,NaN,7882.0,37821.0,37821.0,2020-12
916956,2020-12-14,US,4538.0,35355.0,143.0,8025.0,4709.0,42530.0,2020-12
916957,2020-12-15,US,45876.0,81231.0,290.0,8315.0,47301.0,89831.0,2020-12
916958,2020-12-16,US,154818.0,236049.0,488.0,8803.0,159341.0,249172.0,2020-12
916959,2020-12-17,US,266883.0,502932.0,789.0,9592.0,273552.0,522724.0,2020-12


### Now, we will perform fillna(0) on the 'new_...' columns and groupby + ffill() on the 'cumulative_...' columns.

In [551]:
colsForFillZero = ['new_persons_vaccinated', 'new_persons_fully_vaccinated', 'new_vaccine_doses_administered']
colsForGroupbyFfill = ['cumulative_persons_vaccinated', 'cumulative_persons_fully_vaccinated',
                      'cumulative_vaccine_doses_administered']
df_vacc[colsForFillZero] = df_vacc[colsForFillZero].fillna(0)
df_vacc[colsForGroupbyFfill] = df_vacc.groupby(['location_key'])[colsForGroupbyFfill].apply(lambda x: x.ffill())

/var/folders/8_/94pckqjd52dgrgd_fxmw31280000gp/T/ipykernel_3674/733079642.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vacc[colsForFillZero] = df_vacc[colsForFillZero].fillna(0)
/var/folders/8_/94pckqjd52dgrgd_fxmw31280000gp/T/ipykernel_3674/733079642.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vacc[colsForGroupbyFfill] = df_vacc.groupby(['location_key'])[colsForGroupbyFfill].apply(lambda x: x.ffill())


In [556]:
df_vacc = df_vacc.fillna(0)
fixDataTypesOfColumns(df_vacc, df_vacc.columns.tolist()[2:8], 'float64', 'int64')
df_vacc.head()

Datatypes not matching!


,date,location_key,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered,date_month
916955,2020-12-13,US,30817,30817,0,7882,37821,37821,2020-12
916956,2020-12-14,US,4538,35355,143,8025,4709,42530,2020-12
916957,2020-12-15,US,45876,81231,290,8315,47301,89831,2020-12
916958,2020-12-16,US,154818,236049,488,8803,159341,249172,2020-12
916959,2020-12-17,US,266883,502932,789,9592,273552,522724,2020-12


In [557]:
df_vacc_US = df_vacc.loc[df_vacc['location_key'] == 'US']
df_vacc_states = df_vacc.loc[df_vacc['location_key'] != 'US']
df_vacc_US.reset_index(drop=True, inplace=True)
df_vacc_states.reset_index(drop=True, inplace=True)

### Next, we take a look at df_govResponse

In [559]:
df_govResponse =pd.read_csv("https://storage.googleapis.com/covid19-open-data/v3/oxford-government-response.csv")

In [563]:
df_govResponse = df_govResponse.loc[df_govResponse['location_key'].isin(location_key_US)]

In [565]:
df_govResponse.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48218 entries, 245543 to 293760
Data columns (total 22 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   date                                48218 non-null  object 
 1   location_key                        48218 non-null  object 
 2   school_closing                      48217 non-null  float64
 3   workplace_closing                   48218 non-null  float64
 4   cancel_public_events                48218 non-null  float64
 5   restrictions_on_gatherings          48218 non-null  float64
 6   public_transport_closing            48210 non-null  float64
 7   stay_at_home_requirements           48218 non-null  float64
 8   restrictions_on_internal_movement   48212 non-null  float64
 9   international_travel_controls       48203 non-null  float64
 10  income_support                      48213 non-null  float64
 11  debt_relief                        

### We will filter a few of the columns from df_govResponse.

In [570]:
colsToLeaveOut = ['debt_relief', 'fiscal_measures', 'international_support']
df_govResp = df_govResponse.drop(columns=colsToLeaveOut)
df_govResp.head()

,date,location_key,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings,public_transport_closing,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,income_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,facial_coverings,vaccination_policy,stringency_index
245543,2020-01-01,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
245544,2020-01-02,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
245545,2020-01-03,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
245546,2020-01-04,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
245547,2020-01-05,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [573]:
df_govResp = df_govResp.groupby(['location_key']).apply(lambda x: x.ffill())

In [575]:
df_govResp = df_govResp.fillna(0)
df_govResp.head()

,date,location_key,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings,public_transport_closing,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,income_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,facial_coverings,vaccination_policy,stringency_index
245543,2020-01-01,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
245544,2020-01-02,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
245545,2020-01-03,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
245546,2020-01-04,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
245547,2020-01-05,US,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [576]:
df_govResp['date_month'] = df_govResp['date'].apply(lambda x: '-'.join(x.split('-')[:2]))

### Now, casting columns to correct data types.

In [578]:
df_govResp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48218 entries, 245543 to 293760
Data columns (total 20 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   date                                48218 non-null  object 
 1   location_key                        48218 non-null  object 
 2   school_closing                      48218 non-null  float64
 3   workplace_closing                   48218 non-null  float64
 4   cancel_public_events                48218 non-null  float64
 5   restrictions_on_gatherings          48218 non-null  float64
 6   public_transport_closing            48218 non-null  float64
 7   stay_at_home_requirements           48218 non-null  float64
 8   restrictions_on_internal_movement   48218 non-null  float64
 9   international_travel_controls       48218 non-null  float64
 10  income_support                      48218 non-null  float64
 11  public_information_campaigns       

In [579]:
fixDataTypesOfColumns(df_govResp, df_govResp.columns.tolist()[2:18], 'float64', 'int64')

In [581]:
df_govResp_US = df_govResp.loc[df_govResp['location_key'] == 'US']
df_govResp_states = df_govResp.loc[df_govResp['location_key'] != 'US']

In [582]:
df_govResp_US.reset_index(drop=True, inplace=True)
df_govResp_states.reset_index(drop=True, inplace=True)

# Finally, we will export the clean dataframes as csv files to CovidProj folder on Desktop.

In [601]:
path = '~/Desktop/CovidProj/'
df_epidem_US.to_csv(path+'epidem_US.csv', sep=',', index=False)

In [602]:
df_epidem_states.to_csv(path+'epidem_states.csv', sep=',', index=False)
df_demog_US.to_csv(path+'demog_US.csv', sep=',', index=False)
df_demog_states.to_csv(path+'demog_states.csv', sep=',', index=False)
df_emergD2.to_csv(path+'emerg_declarations.csv', sep=',', index=False)
df_hosp_US.to_csv(path+'hospitalized_US.csv', sep=',', index=False)
df_hosp_states.to_csv(path+'hospitalized_states.csv', sep=',', index=False)
df_mob_US.to_csv(path+'mobility_US.csv', sep=',', index=False)
df_mob_states.to_csv(path+'mobility_states.csv', sep=',', index=False)
df_vacc_US.to_csv(path+'vaccination_US.csv', sep=',', index=False)
df_vacc_states.to_csv(path+'vaccination_states.csv', sep=',', index=False)
df_govResp_US.to_csv(path+'govResp_US.csv', sep=',', index=False)
df_govResp_states.to_csv(path+'govResp_states.csv', sep=',', index=False)

In [604]:
set(df_demog_states.location_key.tolist())

{'US_AK',
 'US_AL',
 'US_AR',
 'US_AZ',
 'US_CA',
 'US_CO',
 'US_CT',
 'US_DC',
 'US_DE',
 'US_FL',
 'US_GA',
 'US_GU',
 'US_HI',
 'US_IA',
 'US_ID',
 'US_IL',
 'US_IN',
 'US_KS',
 'US_KY',
 'US_LA',
 'US_MA',
 'US_MD',
 'US_ME',
 'US_MI',
 'US_MN',
 'US_MO',
 'US_MP',
 'US_MS',
 'US_MT',
 'US_NC',
 'US_ND',
 'US_NE',
 'US_NH',
 'US_NJ',
 'US_NM',
 'US_NV',
 'US_NY',
 'US_OH',
 'US_OK',
 'US_OR',
 'US_PA',
 'US_PR',
 'US_SC',
 'US_SD',
 'US_TN',
 'US_TX',
 'US_UT',
 'US_VA',
 'US_VI',
 'US_VT',
 'US_WA',
 'US_WI',
 'US_WV',
 'US_WY'}

In [608]:
df_mob_states['date_month'] = df_mob_states['date'].apply(lambda x: '-'.join(x.split('-')[:2]))
df_mob_US['date_month'] = df_mob_US['date'].apply(lambda x: '-'.join(x.split('-')[:2]))

/var/folders/8_/94pckqjd52dgrgd_fxmw31280000gp/T/ipykernel_3674/3801502853.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mob_states['date_month'] = df_mob_states['date'].apply(lambda x: '-'.join(x.split('-')[:2]))
/var/folders/8_/94pckqjd52dgrgd_fxmw31280000gp/T/ipykernel_3674/3801502853.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mob_US['date_month'] = df_mob_US['date'].apply(lambda x: '-'.join(x.split('-')[:2]))


In [609]:
df_mob_states.head()

,date,location_key,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential,date_month
3846291,2020-02-15,US_AK,5.0,3.0,14.0,2.0,1.0,0.0,2020-02
3846292,2020-02-16,US_AK,15.0,5.0,29.0,6.0,1.0,-1.0,2020-02
3846293,2020-02-17,US_AK,3.0,-2.0,9.0,-6.0,-33.0,8.0,2020-02
3846294,2020-02-18,US_AK,-2.0,-2.0,1.0,4.0,-2.0,2.0,2020-02
3846295,2020-02-19,US_AK,0.0,-2.0,1.0,4.0,-3.0,2.0,2020-02


In [3]:
# Re-importing the saved files for analysis

import pandas as pd

df = pd.read_csv('~/Desktop/CovidProj/epidem_states.csv', sep=',')

In [5]:
df.head()

,date,location_key,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested,date_month
0,2020-01-22,US_AK,0,0,0,0,0,0,0,0,2020-01
1,2020-01-23,US_AK,0,0,0,0,0,0,0,0,2020-01
2,2020-01-24,US_AK,0,0,0,0,0,0,0,0,2020-01
3,2020-01-25,US_AK,0,0,0,0,0,0,0,0,2020-01
4,2020-01-26,US_AK,0,0,0,0,0,0,0,0,2020-01


In [6]:
df['location_key'] = df['location_key'].apply(lambda x: x.split('_')[1])

In [8]:
df = df[['date', 'location_key', 'new_confirmed', 'new_deceased', 'new_tested']]

In [12]:
df.dtypes

date             object
location_key     object
new_confirmed     int64
new_deceased      int64
new_tested        int64
dtype: object

In [15]:
df.to_csv('~/Desktop/CovidProj/epidem_states_daily.csv', sep=',', index=False)

In [17]:
df.shape

(54218, 5)

In [27]:
df = pd.read_csv('~/Desktop/CovidProj/epidem_US_daily.csv', sep=',')

In [28]:
df.head()

,date,location_key,new_confirmed,new_deceased,new_tested
0,2020-01-01,US,0,0,0
1,2020-01-02,US,0,0,0
2,2020-01-03,US,0,0,0
3,2020-01-04,US,0,0,0
4,2020-01-05,US,0,0,0


In [29]:
df.isna().sum()

date             0
location_key     0
new_confirmed    0
new_deceased     0
new_tested       0
dtype: int64

In [30]:
sum(df['new_deceased'])

988028